In [ ]:
# # This Python 3 environment comes with many helpful analytics libraries installed
# # It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# # For example, here's several helpful packages to load

# import numpy as np # linear algebra
# import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# # Input data files are available in the read-only "../input/" directory
# # For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# # You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# # You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import string
import pickle


#Importing preprocessing modules
!pip install nltk
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import MinMaxScaler
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

#Importing model building modules
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier,AdaBoostClassifier
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.svm import SVC,LinearSVC

#Importing modules for scoring 
from sklearn import metrics

In [ ]:
df_train = pd.read_csv('train.csv')
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train['message'].values[2]

In [ ]:
df_train['tweetid'].nunique()

In [ ]:
df_train.shape

In [ ]:
def clean_title(title): 
    
    pattern_url = r'http[s]?://(?:[A-Za-z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9A-Fa-f][0-9A-Fa-f]))+' #Pattern to remove all hyperlinks
    title = re.sub(pattern_url,'',title)   #Removing all hyperlinks from statement
    #title = re.sub('RT @.*:','',title)    #Removing all preceding RT @ from every post
    title = re.sub('@\w+','',title)  #Removing all  ''@'' twitter handles
    #title = re.sub(r'(\s)#\w+','',title)  ##Removing  all hashtags
    title = re.sub("[^a-zA-Z0-9 ]", "", title) #Extracting Remaining text from  
    title = re.sub(r'[,!?;-]+', '.', title) #Removing any remaining non-characters
    #title = title.replace('RT', '')     #Removing RT{retweet} from posts
    title = re.sub('[%s]' % re.escape(string.punctuation), '', title)
    title = re.sub('\w*\d\w*', '', title)
    title = title.strip().capitalize()   #Removing all remaining whitespaces and Capitalizing all Posts
    return title

In [ ]:
df_train['cleaned_message'] = df_train['message'].apply(clean_title)

In [ ]:
df_train

In [ ]:
df_train.cleaned_message.values[:5]

In [ ]:
term = 'RT @pablorodas: #CLIMATEchange #p2 RT  Trump picks climate change sceptic Scott Pruitt to lead EPA https://t.co/1xb8rrW026 #COP22 https://t…'

In [ ]:
#Function to prepare text for modelling
stemmer = PorterStemmer()
lemmatizer = WordNetLemmatizer()
def model_prep_data(message):
    
    review = message.lower()
    review = review.split()
    
    review = [stemmer.stem(word) for word in review if not word in stopwords.words('english')]
    #review = [word for word in review if len(word) > 1]
    
    
    return ' '.join(review)

In [ ]:
#Creating new column of prepared data to be used for modelling
df_train['model_prep_data'] = df_train['cleaned_message'].apply(model_prep_data)
df_train[['cleaned_message','model_prep_data']]

In [ ]:
# # Creating the Bag of Words vectorizer
# cv = CountVectorizer(stop_words='english', max_features=2500)
# X = cv.fit_transform(df_train.model_prep_data).toarray()
# data_X =  pd.DataFrame(X, columns=cv.get_feature_names())
# data_X.index = df_train.tweetid
# data_X = data_X.T


In [ ]:
# top_dict = {}
# for c in data_X.columns:
#     top = data_X[c].sort_values(ascending=False).head(30)
#     top_dict[c]= list(zip(top.index, top.values))



In [ ]:
# Import the scaler module to scale data
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

# Scale data
X_scaled = scaler.fit_transform(X)

In [ ]:
#Creating a tfidf Vectorizer for modelling
tfidf = TfidfVectorizer(sublinear_tf=True, min_df=5,
                        ngram_range=(1,2)
                       )
# We transform each complaint into a vector
features = tfidf.fit_transform(df_train.model_prep_data).toarray()
labels = df_train.sentiment
model_save_path = "tfidf.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(tfidf,file)
print("Each of the %d complaints is represented by %d features (TF-IDF score of unigrams and bigrams)" %(features.shape))

In [ ]:
# from imblearn.under_sampling import RandomUnderSampler
# from collections import Counter


# rus = RandomUnderSampler(random_state=42, replacement=True)# fit predictor and target variable
# x_rus, y_rus = rus.fit_resample(features, labels)

# print('original dataset shape:', Counter(labels))
# print('Resample dataset shape', Counter(y_rus))

In [ ]:
#Splitting dataset into training and testing set
X_train, X_test, y_train, y_test = train_test_split(features, labels,test_size=0.2, random_state = 42)

# # Import the var thresh model and choose a threshold
# from sklearn.feature_selection import VarianceThreshold
# selector = VarianceThreshold(threshold=0.1)

# # Transform (i.e.: run selection on) the training data
# X_train_vt = selector.fit_transform(X_train)


In [ ]:
#generic function to fit model and return metrics for every algorithm
def boost_models(x):
    '''
    This function receives a model as a parameter, the target for the mordel is then transformed using the quantile transformer 
    to transform it. The model is then trained and tested and the Test set. The errors and accuraciers are then computed
    and returned as a function response
    '''
    
    #Instantiating model to be used
    model = x 
    algoname= x.__class__.__name__
   

    print ('Fitting {:s} model...'.format(algoname))
    run_time = %timeit -q -o  model.fit(X_train, y_train)
    
    
    print ('... predicting')
    y_pred      = model.predict(X_train)
    y_pred_test = model.predict(X_test)
    
    print ('... scoring')
    accuracy  = metrics.accuracy_score(y_train, y_pred)
    precision = metrics.precision_score(y_train, y_pred, average='weighted')
    recall    = metrics.recall_score(y_train, y_pred, average='weighted')    
    f1        = metrics.f1_score(y_train, y_pred, average='weighted')    
    f1_test   = metrics.f1_score(y_test, y_pred_test, average='weighted')   
    
    #Extracting the name of the algorithm
    
    return algoname, accuracy, precision, recall, f1, f1_test, run_time.best

# # Instantiating base models that will be required by the adaboost regression model
# d_tree = DecisionTreeRegressor(max_depth= 7)
# lgbr = lgbm.LGBMRegressor()


classifiers = [
    LogisticRegression(max_iter=10000, multi_class='ovr', solver='liblinear'),
    MultinomialNB(),
    KNeighborsClassifier(2),
    LinearSVC(penalty='l2',multi_class='ovr'),
#    SVC(kernel="linear", C=0.025),
#    SVC(gamma=2, C=1),
    #DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=7))
]
score=[]

for algo in classifiers:
    score.append(boost_models(algo))

 #Collate all scores in a table
results = pd.DataFrame(score, columns=['Model', 'Accuracy', 'Precision', 'Recall', 'F1 Train', 'F1 Test', 'Train Time'])
results.set_index('Model')

In [ ]:
# from imblearn.over_sampling import RandomOverSampler

# ros = RandomOverSampler(random_state=42)

# # fit predictor and target variable
# x_ros, y_ros = ros.fit_resample(features, labels)

# print('Original dataset shape', Counter(labels))
# print('Resample dataset shape', Counter(y_ros))

In [ ]:
n_neighbors = 3 # <--- change this number to play around with how many nearest neighbours to look for.

knn = KNeighborsClassifier(n_neighbors)
# Fit the model 
knn.fit(X_train, y_train)
#y_pred=nv_model.predict(X_test)
print(f'Trainging set prediction performance is {knn.score(X_train, y_train) }')
print(f'Testing set prediction performance is {knn.score(X_test, y_test) }')
model_save_path = "knn.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(knn,file)

In [ ]:
from sklearn import metrics
ks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 20, 50, 100]

results = []

for k in ks:
    print('Fitting KNN model with k = {:d}'.format(k))
    knn = KNeighborsClassifier(k)
    run_time = %timeit -q -o knn.fit(X_train, y_train)
    
    # predicting
    y_pred = knn.predict(X_train)   
    y_pred_test = knn.predict(X_test)
    
    # scoring
    accuracy  = metrics.accuracy_score(y_train, y_pred)
    precision = metrics.precision_score(y_train, y_pred,average='weighted')
    recall    = metrics.recall_score(y_train, y_pred,average='weighted')    
    f1        = metrics.f1_score(y_train, y_pred,average='weighted')    
    f1_test   = metrics.f1_score(y_test, y_pred_test,average='weighted')    
    
    # save the results 
    results.append([k, accuracy, precision, recall, f1, f1_test, run_time.best])
    
results = pd.DataFrame(results, columns=['KNN', 'Accuracy', 'Precision', 'Recall', 'F1 Train', 'F1 Test', 'Train Time'])
results.set_index('K', inplace= True)

In [ ]:
results = pd.DataFrame(results, columns=['KNN', 'Accuracy', 'Precision', 'Recall', 'F1 Train', 'F1 Test', 'Train Time'])
results.set_index('KNN', inplace= True)

In [ ]:
results

In [ ]:
# estimators = [
# ...     ('rf', MultinomialNB()),
# ...     ('svr', make_pipeline(StandardScaler(),
# ...                           LinearSVC(ax_iter=10000random_state=42)))
# ]

In [ ]:
# clf = StackingClassifier(
# ...     estimators=estimators, final_estimator=LogisticRegression(max_iter=10000, multi_class='ovr', solver='liblinear'),
# )

# clf.fit(X_train,y_train)
# clf.score(X_test,y_test)

In [ ]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from numpy import  mean

model = DecisionTreeClassifier()


over = SMOTE(sampling_strategy={-1: 3000, 0:4000,2:5000})
under = RandomUnderSampler(sampling_strategy={1:6000})
steps = [('over', over), ('under', under), ('model', model)]

pipeline = Pipeline(steps=steps)

cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=1)
scores = cross_val_score(pipeline, features, labels, scoring='roc_auc', cv=cv, n_jobs=-1)
print('Mean ROC AUC: %.3f' % mean(scores))

In [ ]:
svm_model = LinearSVC(penalty='l2',multi_class='ovr')
svm_model.fit(X_train, y_train)

print(f'Trainging set prediction performance is {svm_model.score(X_train, y_train) }')
print(f'Testing set prediction performance is {svm_model.score((X_test), y_test) }')
model_save_path = "svm.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(svm_model,file)

In [ ]:
#Creating a baseline prediction to be used to access model performance
acc_baseline = y_train.value_counts(normalize=True).max()
print("Baseline Accuracy:", round(acc_baseline, 2))

In [ ]:
# Training model using Naive bayes classifier
from sklearn.naive_bayes import MultinomialNB
NV_model = MultinomialNB()
NV_model.fit(X_train, y_train)

#y_pred=nv_model.predict(X_test)
print(f'Trainging set prediction performance is {NV_model.score(X_train, y_train) }')
print(f'Testing set prediction performance is {NV_model.score(X_test, y_test) }')
model_save_path = "nb.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(NV_model,file)

In [ ]:
import xgboost as xgb
model = xgb.XGBClassifier()
model.fit(X_train, y_train)
print(); print(model.score(X_train, y_train))

In [ ]:
#Logistic regression model for training 
lm_model = LogisticRegression(multi_class='ovr', solver='liblinear')
lm_model.fit(X_train, y_train)

print(f'Trainging set prediction performance is {lm_model.score(X_train, y_train)}')
print(f'Testing set prediction performance is {lm_model.score(X_test,y_test) }')
model_save_path = "lrm.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(lm_model,file)

In [ ]:
from sklearn.ensemble import BaggingClassifier
# from sklearn.neighbors import KNeighborsClassifier
bagging = BaggingClassifier(LogisticRegression(max_iter=10000,multi_class='ovr', solver='liblinear'),
                            max_samples=0.5, 
                            max_features=0.5
                           )

In [ ]:
bagging.fit(X_train,y_train)
model_save_path = "bc.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(bagging,file)
bagging.score(X_test,y_test)

In [ ]:
#random forest model for model training 
rf_model = RandomForestClassifier(n_estimators=1000,max_depth=7)
rf_model.fit(X_train,y_train)

print(f'Trainging set prediction performance is {rf_model.score(X_train, y_train) }')
print(f'Testing set prediction performance is {rf_model.score(X_test, y_test) }')
model_save_path = "rf.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(rf_model,file)

In [ ]:
from imblearn.ensemble import BalancedBaggingClassifier
from sklearn.tree import DecisionTreeClassifier
#Create an instance
classifier = BalancedBaggingClassifier(base_estimator=DecisionTreeClassifier(),
                                sampling_strategy='not majority',
                                replacement=False,
                                random_state=42)
classifier.fit(X_train, y_train)
print(f'Trainging set prediction performance is {classifier.score(X_train, y_train) }')
print(f'Testing set prediction performance is {classifier.score(X_test, y_test) }')
model_save_path = "balbc.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(classifier,file)

In [ ]:
classifier.score(X_test, y_test) 

In [ ]:
from sklearn.metrics import f1_score

f1_score(y_test, y_pred, average='weighted')

In [ ]:
from sklearn import metrics
print(metrics.classification_report(y_test, preds))

In [ ]:

Dt = DecisionTreeClassifier()

ada_model = AdaBoostClassifier(base_estimator=Dt)
ada_model.fit(X_train, y_train)

ada_model.predict(X_test)
print(f'Trainging set prediction performance is {ada_model.score(X_train, y_train) }')
print(f'Testing set prediction performance is {ada_model.score(X_test, y_test) }')
model_save_path = "ada.pkl"
with open(model_save_path,'wb') as file:
    pickle.dump(ada_model,file)

In [ ]:
#Setting up test data for submission
df_test = pd.read_csv('/kaggle/input/edsa-climate-change-belief-analysis-2022/test.csv')
df_test['cleaned_message'] = df_test['message'].apply(clean_title)
df_test['model_prep_data'] = df_test['cleaned_message'].apply(model_prep_data)
test_X = tfidf.transform(df_test.model_prep_data).toarray()

In [ ]:
#Making prediction for submission
new_pred = clf.predict(test_X)
df_test['sentiment'] = new_pred
df_test.rename(columns={'pred':'sentiment'},inplace=True)
sub = df_test[['tweetid','sentiment']]
sub = sub.set_index('tweetid')
#sub.to_csv('Submission17.csv')

In [ ]:
#sub.to_csv('Submission12.csv')